In [2]:
from dotenv import load_dotenv

_ = load_dotenv()

In [3]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage

memory = SqliteSaver.from_conn_string(":memory:")

In [4]:
# Define the state
class IQAState(TypedDict):
    task: str
    course_structure: str
    evaluation_methods: List[str]
    evaluation_level: str
    content: List[str]
    framework_results: List[dict]
    final_summary: str
    evaluation_round: int
    max_rounds: int

In [5]:
from langchain_openai import ChatOpenAI
import os
llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))

In [6]:
# Prompts
CONFIRMATION_PROMPT = """You are an instructional quality evaluator. 
The user has uploaded a course with the following structure and categories:
{course_structure}

Please confirm if the uploaded content matches the expected scope of the course. 
Confirm the categories and any files (e.g., PDFs, videos, audio) in the course package. 
If there are discrepancies or missing files, ask the user to re-upload the content or correct the errors."""

EVALUATION_FRAMEWORKS_PROMPT = """You are an instructional design expert.
Based on the course provided by the user, suggest one or two proven frameworks or methods that will be used to evaluate the instructional quality of the course. 
Make sure to provide a clear description of each framework and explain how it applies to the specific course content and its goals."""

DEPTH_ACCURACY_PROMPT = """You are an instructional quality evaluator. 
Ask the user the level of depth and accuracy they want from the evaluation. 
Provide options such as 'Basic Overview', 'In-Depth Analysis', or 'Detailed Critique' and explain what each level means in terms of the feedback they will receive."""

PROCESS_CONFIRMATION_PROMPT = """You are an instructional quality evaluator. 
The user has selected the following evaluation method(s): 
{evaluation_methods}

Please confirm the process that will be followed and outline any additional steps the user needs to know about.
Reassure the user that the evaluation will begin shortly, and they will receive both quantitative ratings and qualitative feedback."""

REPORT_GENERATION_PROMPT = """You are an instructional quality evaluator. 
Generate a final report for the user based on the course evaluation. 
The report should include:
1. A numerical rating of the overall course quality (0-10 scale).
2. Scores for each of the evaluation criteria (e.g., content clarity, instructional methods, engagement, etc.).
3. A summary of findings and actionable feedback for improving the course.
Ensure that the report is clear, concise, and professional."""

FEEDBACK_RECOMMENDATIONS_PROMPT = """You are an instructional design expert.
Based on the evaluation, generate detailed feedback and recommendations for the user to improve their course. 
Cover aspects such as content structure, clarity, engagement, and instructional design best practices.
Provide specific suggestions for each area and prioritize improvements that will have the greatest impact on the course quality."""

FOLLOWUP_EVALUATION_PROMPT = """You are an instructional quality evaluator.
The user has revised their course based on your initial feedback.
Evaluate the updated course by applying the same criteria and frameworks used in the first evaluation.
Provide a comparison of the original and updated versions and comment on any improvements or areas still needing attention."""

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from file_processor import FileProcessor

file_processor = FileProcessor()

load_doc = file_processor.load_pdf("files/RFP DATA CENTER.doc-1.pdf")

map_prompt = ChatPromptTemplate.from_messages(
    [("system", "Write a concise summary of the following:\\n\\n{context}")]
)

In [8]:
from langchain import hub

map_prompt = hub.pull("rlm/map-prompt")

c:\Users\Area-51m\anaconda3\envs\evaToolEnv\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [9]:
# Also available via the hub: `hub.pull("rlm/reduce-prompt")`
reduce_template = """
The following is a set of summaries:
{docs}
Take these and distill it into a final, consolidated summary
of the main themes.
"""

reduce_prompt = ChatPromptTemplate([("human", reduce_template)])

In [10]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=2000, chunk_overlap=200
)
split_docs = text_splitter.split_documents(load_doc)
print(f"Generated {len(split_docs)} documents.")

Generated 67 documents.


In [11]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Define prompt
prompt = ChatPromptTemplate.from_messages(
    [("system", "Write a concise summary of the following:\\n\\n{context}")]
)

# Instantiate chain
chain = create_stuff_documents_chain(llm, prompt)

# Invoke chain
result = chain.invoke({"context": split_docs})
print(result)

The Ministry of Information, Communication and Information Technology of the United Republic of Tanzania has issued a Request for Proposal (RFP) for consultancy services to establish a Tier-III Data Center in Zanzibar. This initiative is part of the government's efforts to enhance ICT infrastructure, promote citizen participation, and facilitate e-services. The RFP invites proposals from shortlisted consulting firms to conduct a feasibility study, design the data center, and provide technical specifications, including a Bill of Quantities (BoQ) and estimated budget.

Key objectives include assessing current data center needs, ensuring the architectural and environmental integrity of the proposed facility, and providing recommendations for project management and a suitable business model. The consultancy work is expected to span four months, and firms will be evaluated based on their technical and financial proposals under a Quality and Cost Based Selection (QCBS) method.

The RFP outli